In [1]:
import sys
sys.path.append('c:/Users/iwosz/PythonProjects/CompilationTheory/') 

In [2]:
from lab3.MyParserAST import ParserMatrixAST
from lab3.AST import *
from lab1.MyLexer import MyLexer as Scanner
from lab3.TreePrinter import TreePrinter
from lab4.TypeChecker import TypeChecker
from lab5.visit import *
from lab5.Interpreter import Interpreter

Basic tests

In [3]:
text ='''A = eye(5); # create 5x5 matrix filled with zeros
B = ones(5);  # create 5x5 matrix filled with ones
D = A.+B;   # add element-wise A with transpose of B
x = [1,2,3,4,5]; # create vector x
b = zeros(5,5);
for j = 0:4 
    print j;
print D;
print D[1];
'''
par = ParserMatrixAST()
ast = par.parse(Scanner().tokenize(text))
treePrinter = TreePrinter()
typeChecker = TypeChecker()

if ast is not None:
    ast.printTree(0)
    print("Type checking errors: ", typeChecker.visit(ast))
    interpreter = Interpreter()
    interpreter.visit(ast)

A
|	=
|	|	eye
|	|	|	5
B
|	=
|	|	ones
|	|	|	5
D
|	=
|	|	A
|	|	|	.+
|	|	|	|	B
x
|	=
|	|	VECTOR
|	|	|	1
|	|	|	2
|	|	|	3
|	|	|	4
|	|	|	5
b
|	=
|	|	zeros
|	|	|	5
|	|	|	5
FOR
|	j
|	RANGE
|	|	0
|	|	4
|	PRINT
|	|	j
PRINT
|	D
PRINT
|	REF
|	|	D
|	|	1
Symbol "D" not found, Scope: global {'A': 'vector', 'B': 'vector', 'x': 'vector', 'b': 'vector'}
Type checking errors:  None
0
1
2
3
4
[2, 2, 2, 2, 2]
2


Tests from files

In [7]:
test_nums = [i for i in range(1,7)]
print(test_nums)
# test_nums = [2]

[1, 2, 3, 4, 5, 6]


In [8]:
def print_data_with_line_nrs(data):
    for i, line in enumerate(data.split('\n')):
        print('{:>3}: {}'.format(i+1, line))

In [9]:
for n in test_nums:
    print(f"Test nr {n}:")
    filename = f"tests/test{n}.m"
    with open(filename, 'r') as f:
        data = f.read()

    print_data_with_line_nrs(data)
    parser = ParserMatrixAST()
    ast = parser.parse(Scanner().tokenize(data))
    print("AST tree is: ")
    ast.printTree(0)
    typeChecker = TypeChecker()
    print(f"Type checking errors: {typeChecker.visit(ast)}")
    
    print("Interpreter: ")
    interpreter = Interpreter()
    interpreter.visit(ast)
    print("\n")

Test nr 1:
  1: a = 0;
  2: b = 1;
  3: while (b < 1000) {
  4:     print b;
  5:     b += a;
  6:     a = b - a;
  7: }
  8: 
AST tree is: 
a
|	=
|	|	0
b
|	=
|	|	1
WHILE
|	b
|	|	<
|	|	|	1000
|	PRINT
|	|	b
|	|	b
|	|	+=
|	|	|	a
|	|	a
|	|	=
|	|	|	b
|	|	|	|	-
|	|	|	|	|	a
Type checking errors: None
Interpreter: 
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


Test nr 2:
  1: A = eye(3);
  2: B = ones(3);
  3: C = zeros(3);
  4: C = A .+ B;
  5: print C;
  6: D = zeros(3, 4);
  7: D[0, 0] = 42;
  8: D[1, 2] = 7;
  9: print D;
 10: print D[1, 2];
 11: 
AST tree is: 
A
|	=
|	|	eye
|	|	|	3
B
|	=
|	|	ones
|	|	|	3
C
|	=
|	|	zeros
|	|	|	3
C
|	=
|	|	A
|	|	|	.+
|	|	|	|	B
PRINT
|	C
D
|	=
|	|	zeros
|	|	|	3
|	|	|	4
REF
|	D
|	0
|	0
|	=
|	|	42
REF
|	D
|	1
|	2
|	=
|	|	7
PRINT
|	D
PRINT
|	REF
|	|	D
|	|	1
|	|	2
Type checking errors: None
Interpreter: 
[2, 2, 2]
[[42, 0, 0], [0, 0, 0], [0, 7, 0], [0, 0, 0]]
7


Test nr 3:
  1: pi = 0.0;
  2: n = 1;
  3: for i = 1:100000 {
  4:     pi += 4.0 / n - 4.0 / (n 